In [17]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [18]:
from models import TransformerModel
import torch
import yaml
from data import get_tgt_str, build_vocab, id2token

In [59]:
config_path = "/home/takeru/AlphaSymbol/train_script/config_20241202235904.yaml"
state_dict = torch.load("/home/takeru/AlphaSymbol/train_script/model_30000epochs_20241202235904.pth")

with open(config_path, "r") as f:
    config = yaml.safe_load(f)

src_max_len = config["src_max_len"]
tgt_max_len = config["tgt_max_len"]
src_vocab = config["src_vocab"]
tgt_vocab = config["tgt_vocab"]

model = TransformerModel(len(src_vocab), len(tgt_vocab), src_max_len, tgt_max_len)
model.eval()

inputs = [(0,), (1,), (2,), (3,), (4,), (5,), (6,), (7,), (8,), (9,)]
# outputs = [2, 3, 4, 5, 6, 7, 8, 9, 10, 11]
outputs = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
# outputs = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
src_str = get_tgt_str(inputs, outputs)
src_tensor = torch.tensor([src_vocab[i] for i in src_str])
src_tensor = torch.cat([src_tensor, torch.tensor([src_vocab["<pad>"] for _ in range(src_max_len - src_tensor.shape[0])])])
src_tensor = src_tensor.unsqueeze(0)
out_str = torch.tensor(tgt_vocab["<sos>"]).reshape((1, ))
out_str = out_str.unsqueeze(0)

with torch.no_grad():
    for i in range(tgt_max_len):
        model.load_state_dict(state_dict)
        model.eval()
        pred = model(src_tensor, out_str)
        next_c = pred[0, -1, :]
        max_values, max_indices = torch.max(next_c, axis=0)
        out_str = torch.cat((out_str, max_indices.reshape(1, 1)), axis=1)
        if max_indices == tgt_vocab["<eos>"]:
            break

for id in out_str[0]:
    print(id2token(tgt_vocab, id))

/tmp/ipykernel_20380/1392261157.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load("/home/takeru/AlphaSymbol/train_script/model_30000epochs_20241202

<sos>
Z
)
<eos>
